In [1]:

from binarybeech.binarybeech import *
import binarybeech.utils as utils


In [2]:

df = pd.read_csv("data/titanic.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic),:]
    

In [3]:

gbt_titanic = GradientBoostedTree(df_titanic,"Survived",learning_rate=0.5)


In [4]:
gbt_titanic.X_names

['Pclass',
 'Name',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

In [10]:
it=gbt_titanic._initial_tree()
utils.print_tree(it.tree)


A tree with 1 leafs was created
0.3954887218045112



In [6]:
it.tree.root

In [7]:

gbt_titanic.train(30)


A tree with 1 leafs was created
Norm of pseudo-residuals: 12.608983550049285
A tree with 8 leafs was created
5.37	 0.4498
Norm of pseudo-residuals: 10.214219622937769
A tree with 8 leafs was created
5.55	 0.4249
Norm of pseudo-residuals: 9.490115239257046
A tree with 8 leafs was created
4.78	 0.4172
Norm of pseudo-residuals: 9.293283895079664
A tree with 8 leafs was created
5.11	 0.4131
Norm of pseudo-residuals: 9.227818563955102
A tree with 8 leafs was created
3.82	 0.4062
Norm of pseudo-residuals: 9.141015771176786
A tree with 8 leafs was created
3.04	 0.3988
Norm of pseudo-residuals: 9.046623206781247
A tree with 6 leafs was created
4.05	 0.3986
Norm of pseudo-residuals: 9.030501705028126
A tree with 8 leafs was created
2.84	 0.3971
Norm of pseudo-residuals: 9.009632144983836
A tree with 8 leafs was created
3.26	 0.3906
Norm of pseudo-residuals: 8.948316718986941
A tree with 7 leafs was created
3.69	 0.3886
Norm of pseudo-residuals: 8.910001605397674
A tree with 5 leafs was created


In [8]:

gbt_titanic._predict(df_titanic.iloc[0])

0.9420404735022848

In [9]:

gbt_titanic.validate()


{'precision': array([0.90547264, 0.78326996]),
 'recall': array([0.86460808, 0.8442623 ]),
 'F-score': 0.848595962718778,
 'accuracy': 0.8571428571428571}